In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, average_precision_score, roc_auc_score
import matplotlib.pyplot as plt
import transformers as trfm
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

## Helper functions

In [2]:
def touch_dir(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        print(f"Created directory {dirname}.")
    else:
        print(f"Directory {dirname} already exists.")

In [3]:
def encode_qa(questions, answers, tokenizer, chunk_size=256, maxlen=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(questions), chunk_size)):
        q_chunk = questions[i:i+chunk_size].tolist()
        a_chunk = answers[i:i+chunk_size].tolist()
        text_chunk = list(zip(q_chunk, a_chunk))
        
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [4]:
def build_model(transformer, max_len=256):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_ids = L.Input(shape=(max_len, ), dtype=tf.int32)
    
    x = transformer(input_ids)[0]
    x = x[:, 0, :]
    x = L.Dense(1, activation='sigmoid', name='sigmoid')(x)
    
    # BUILD AND COMPILE MODEL
    model = Model(inputs=input_ids, outputs=x)
    model.compile(
        loss='binary_crossentropy', 
        metrics=['accuracy'], 
        optimizer=Adam(lr=1e-5)
    )
    
    return model

In [5]:
def save_model(model, sigmoid_dir='transformer', transformer_dir='transformer'):
    """
    Special function to load a keras model that uses a transformer layer
    """
    os.makedirs(transformer_dir, exist_ok=True)
    os.makedirs(sigmoid_dir, exist_ok=True)
    
    transformer = model.layers[1]
    transformer.save_pretrained(transformer_dir)
    
    sigmoid_path = os.path.join(sigmoid_dir,'sigmoid.pickle')
    sigmoid = model.get_layer('sigmoid').get_weights()
    pickle.dump(sigmoid, open(sigmoid_path, 'wb'))

    
def load_model(sigmoid_dir='transformer', transformer_dir='transformer', 
               architecture="electra", max_len=None):
    """
    Special function to load a keras model that uses a transformer layer
    """
    sigmoid_path = os.path.join(sigmoid_dir,'sigmoid.pickle')
    
    if architecture == 'electra':
        transformer = trfm.TFElectraModel.from_pretrained(transformer_dir)
    else:
        transformer = trfm.TFAutoModel.from_pretrained(transformer_dir)
    model = build_model(transformer, max_len=max_len)
    
    sigmoid = pickle.load(open(sigmoid_path, 'rb'))
    model.get_layer('sigmoid').set_weights(sigmoid)
    
    return model

## TPU Configs

In [6]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [7]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 5
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
MODEL = 'distilbert-base-multilingual-cased'

## Load data

In [8]:
df = pd.read_csv('/kaggle/input/healthtap-eda/healthtap_qa_pairs.csv')

In [9]:
wrong1 = (
    df[['qid', 'answer']]
    .drop_duplicates("qid", keep="first").copy()
    .rename(columns={'qid': 'wrong_qid_1', 'answer': 'wrong_answer_1'})
)

wrong2 = (
    df[['qid', 'answer']]
    .drop_duplicates("qid", keep="last").copy()
    .rename(columns={'qid': 'wrong_qid_2', 'answer': 'wrong_answer_2'})
)

In [10]:
df = df.merge(wrong1, on='wrong_qid_1', how='left').merge(wrong2, on='wrong_qid_2', how='left')

# Remove this when actually running the real model
df = df.sample(n=500000, random_state=0)

## Bert tokenizer

In [11]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Tokenizer(vocabulary_size=119547, model=BertWordPiece, add_special_tokens=True, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=False, wordpieces_prefix=##)

## Convert text to matrices

In [12]:
correct_ids = encode_qa(df.question, df.answer, fast_tokenizer, maxlen=MAX_LEN)
wrong_ids = encode_qa(df.question, df.wrong_answer_1, fast_tokenizer, maxlen=MAX_LEN)

In [13]:
input_ids = np.concatenate([correct_ids, wrong_ids])

labels = np.concatenate([
    np.ones(correct_ids.shape[0]),
    np.zeros(wrong_ids.shape[0])
]).astype(np.int32)

## Train test split

In [14]:
train_idx, test_idx = train_test_split(
    np.arange(input_ids.shape[0]), 
    test_size=0.3, 
    random_state=0
)

valid_idx, test_idx = train_test_split(
    test_idx, 
    test_size=0.5, 
    random_state=1
)

In [15]:
train_ids = input_ids[train_idx]
valid_ids = input_ids[valid_idx]
test_ids = input_ids[test_idx]

train_labels = labels[train_idx]
valid_labels = labels[valid_idx]
test_labels = labels[test_idx]

## Build datasets objects

In [16]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_ids, train_labels))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_ids, valid_labels))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_ids)
    .batch(BATCH_SIZE)
)

## Modeling

In [17]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 192, 768),)       134734080 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
sigmoid (Dense)              (None, 1)                 769       
Total params: 134,734,849
Trainable params: 134,734,849
Non-trainable params: 0
_________________________________________________________________
CPU times: user 31.5 s, sys: 11.1 s, total: 42.6 s
Wall time: 47.4 s


### Train model

In [18]:
n_steps = train_labels.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Train for 5468 steps, validate for 1172 steps
Epoch 1/5
5468/5468 [==============================] - 636s 116ms/step - loss: 0.4082 - accuracy: 0.8021 - val_loss: 0.4278 - val_accuracy: 0.7963
Epoch 4/5
5468/5468 [==============================] - 635s 116ms/step - loss: 0.3741 - accuracy: 0.8225 - val_loss: 0.4285 - val_accuracy: 0.8023
Epoch 5/5
3909/5468 [====================>.........] - ETA: 2:48 - loss: 0.3459 - accuracy: 0.8384

In [19]:
hist_df = pd.DataFrame(train_history.history)
hist_df.to_csv('train_history.csv')
hist_df

,loss,accuracy,val_loss,val_accuracy
0,0.522774,0.727734,0.474639,0.760213
1,0.449358,0.777724,0.436521,0.788413
2,0.408246,0.802123,0.427847,0.796347
3,0.374055,0.822474,0.428476,0.802320
4,0.342922,0.839745,0.431538,0.806593


In [20]:
save_model(model)

## Eval

In [21]:
with strategy.scope():
    model = load_model(max_len=MAX_LEN, architecture='distilbert')

In [22]:
y_score = model.predict(test_dataset, verbose=1).squeeze()
y_pred = y_score.round().astype(int)
print("AP:", average_precision_score(test_labels, y_score))
print("ROC AUC:", roc_auc_score(test_labels, y_score))
print(classification_report(test_labels, y_pred))

1172/1172 [==============================] - 73s 63ms/step
AP: 0.8969144338597053
ROC AUC: 0.8940913075007604
              precision    recall  f1-score   support

           0       0.79      0.82      0.81     75248
           1       0.81      0.78      0.80     74752

    accuracy                           0.80    150000
   macro avg       0.80      0.80      0.80    150000
weighted avg       0.80      0.80      0.80    150000

